In [0]:
-- Create SQL Procedure for ADS_RDS_EVENT_TYPES_MEP ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_RDS_EVENT_TYPES_MEP(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for ADS RDS Event Types MEP - converted from notebook format'
AS
BEGIN
    -- Declare local variables
    DECLARE dif_table_name STRING DEFAULT 'gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MEP_ADS_MAP_SCD_DIFF';
    BEGIN
        CALL gap_catalog.log.start_performance_test(
        test_case_name => 'ADS_RDS_Q',
        step_name => 'ADS_RDS_EVENT_TYPES_MEP',
        psource_sys_origin => 'RDS_MEPOPERATIONTYPE',
        process_key => p_process_key,
        target_table => 'EVENT_TYPES',
        p_load_date => p_load_date
        );
        -- Step 1: Truncate XC Table
        TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MEP;

        -- Step 2: Fill XC Table
        INSERT INTO gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MEP 
        WITH C_0FILTERALQDR4RF0LLIP1736450PD5NP4 AS (
            SELECT
                RDS_MEPOPERATIONTYPE.id AS ID,
                RDS_MEPOPERATIONTYPE.value AS VALUE
            FROM gap_catalog.ads_etl_owner.DLK_ADS_LOV_RDS_MEPOPERATIONTYPE RDS_MEPOPERATIONTYPE
            WHERE RDS_MEPOPERATIONTYPE.sys = 'Brasil'
                AND RDS_MEPOPERATIONTYPE.lang = 'CZ'
                AND CAST(from_utc_timestamp(SYS_EFFECTIVE_DATE, 'Europe/Prague') AS DATE) = CAST(p_load_date AS DATE)
        )
        SELECT 
            FILTER_A.ID as EVETP_SOURCE_ID,
            'RDS' as EVETP_SOURCE_SYSTEM_ID,
            'RDS_MEPOPERATIONTYPE' as EVETP_SOURCE_SYS_ORIGIN,
            FILTER_A.VALUE as EVETP_DESC,
            FILTER_A.ID as EVETP_EVENT_TABLE_NAME,
            'XNA' as EVETP_TARGET,
            'XNA' as EVETP_DATA_PATH,
            'N' as EVETP_DELETED_FLAG
        FROM C_0FILTERALQDR4RF0LLIP1736450PD5NP4 FILTER_A
        WHERE (1=1);

        -- Step 3: Cleanup and Create DIFF Table
        DROP TABLE IF EXISTS gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MEP_ADS_MAP_SCD_DIFF;

        CREATE TABLE gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MEP_ADS_MAP_SCD_DIFF (
            tech_del_flg CHAR(1),
            tech_new_rec CHAR(1),
            tech_rid VARCHAR(255),
            EVETP_KEY INTEGER,
            EVETP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
            EVETP_SOURCE_ID VARCHAR(400),
            EVETP_SOURCE_SYSTEM_ID VARCHAR(120),
            EVETP_SOURCE_SYS_ORIGIN VARCHAR(120),
            EVETP_DESC VARCHAR(4000),
            EVETP_EVENT_TABLE_NAME VARCHAR(120),
            EVETP_TARGET VARCHAR(120),
            EVETP_DATA_PATH VARCHAR(120)
        );

        -- Step 4: Populate DIFF Table - New/Updated Records
        INSERT INTO gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MEP_ADS_MAP_SCD_DIFF (
            tech_del_flg,
            tech_new_rec,
            tech_rid,
            EVETP_KEY,
            EVETP_SOURCE_ID,
            EVETP_SOURCE_SYSTEM_ID,
            EVETP_SOURCE_SYS_ORIGIN,
            EVETP_DESC,
            EVETP_EVENT_TABLE_NAME,
            EVETP_TARGET,
            EVETP_DATA_PATH
        )
        SELECT 
            'N' as tech_del_flg, 
            CASE WHEN trg.EVETP_SOURCE_ID IS NULL THEN 'Y' ELSE 'N' END as tech_new_rec,
            trg.rid as tech_rid,
            trg.EVETP_KEY, 
            src.EVETP_SOURCE_ID, 
            src.EVETP_SOURCE_SYSTEM_ID, 
            src.EVETP_SOURCE_SYS_ORIGIN, 
            src.EVETP_DESC, 
            src.EVETP_EVENT_TABLE_NAME, 
            src.EVETP_TARGET, 
            src.EVETP_DATA_PATH
        FROM (
            SELECT 
                EVETP_SOURCE_ID, 
                EVETP_SOURCE_SYSTEM_ID, 
                EVETP_SOURCE_SYS_ORIGIN, 
                EVETP_DESC, 
                EVETP_EVENT_TABLE_NAME, 
                EVETP_DELETED_FLAG, 
                EVETP_TARGET, 
                EVETP_DATA_PATH
            FROM gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MEP xc
            WHERE EVETP_SOURCE_SYS_ORIGIN = 'RDS_MEPOPERATIONTYPE'
        ) src 
        LEFT JOIN (
            SELECT 
                CONCAT(evetp_key, '.', evetp_valid_from) as rid, 
                t.* 
            FROM gap_catalog.ads_owner.EVENT_TYPES t
            WHERE EVETP_CURRENT_FLAG = 'Y'
                AND EVETP_VALID_TO = DATE('3000-01-01')
                AND EVETP_SOURCE_SYS_ORIGIN = 'RDS_MEPOPERATIONTYPE'      
        ) trg
        ON trg.EVETP_SOURCE_ID = src.EVETP_SOURCE_ID
            AND trg.EVETP_SOURCE_SYSTEM_ID = src.EVETP_SOURCE_SYSTEM_ID
            AND trg.EVETP_SOURCE_SYS_ORIGIN = src.EVETP_SOURCE_SYS_ORIGIN
            AND trg.EVETP_VALID_TO = DATE('3000-01-01') 
        WHERE (
            COALESCE(src.EVETP_DESC, '') != COALESCE(trg.EVETP_DESC, '') OR
            COALESCE(src.EVETP_EVENT_TABLE_NAME, '') != COALESCE(trg.EVETP_EVENT_TABLE_NAME, '') OR
            COALESCE(src.EVETP_DELETED_FLAG, '') != COALESCE(trg.EVETP_DELETED_FLAG, '') OR
            COALESCE(src.EVETP_TARGET, '') != COALESCE(trg.EVETP_TARGET, '') OR
            COALESCE(src.EVETP_DATA_PATH, '') != COALESCE(trg.EVETP_DATA_PATH, '') OR
            trg.EVETP_SOURCE_ID IS NULL
        );

        -- Step 5: Populate DIFF Table - Deleted Records
        INSERT INTO gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MEP_ADS_MAP_SCD_DIFF (
            tech_del_flg,
            tech_new_rec,
            tech_rid,
            EVETP_KEY,
            EVETP_SOURCE_ID,
            EVETP_SOURCE_SYSTEM_ID,
            EVETP_SOURCE_SYS_ORIGIN,
            EVETP_DESC,
            EVETP_EVENT_TABLE_NAME,
            EVETP_TARGET,
            EVETP_DATA_PATH
        )
        SELECT 
            'Y' as tech_del_flg, 
            'N' as tech_new_rec, 
            trg.rid as tech_rid, 
            trg.EVETP_KEY, 
            trg.EVETP_SOURCE_ID, 
            trg.EVETP_SOURCE_SYSTEM_ID, 
            trg.EVETP_SOURCE_SYS_ORIGIN, 
            trg.EVETP_DESC, 
            trg.EVETP_EVENT_TABLE_NAME, 
            trg.EVETP_TARGET, 
            trg.EVETP_DATA_PATH
        FROM (
            SELECT 
                CONCAT(evetp_key, '.', evetp_valid_from) as rid, 
                t.* 
            FROM gap_catalog.ads_owner.EVENT_TYPES t
            WHERE EVETP_CURRENT_FLAG = 'Y' 
                AND EVETP_DELETED_FLAG = 'N'
                AND EVETP_VALID_TO = DATE('3000-01-01')
                AND EVETP_SOURCE_SYS_ORIGIN = 'RDS_MEPOPERATIONTYPE'      
        ) trg 
        LEFT JOIN (
            SELECT 
                EVETP_SOURCE_ID, 
                EVETP_SOURCE_SYSTEM_ID, 
                EVETP_SOURCE_SYS_ORIGIN, 
                DATE('3000-01-01') as EVETP_VALID_TO
            FROM gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MEP xc
            WHERE EVETP_SOURCE_SYS_ORIGIN = 'RDS_MEPOPERATIONTYPE'
        ) src
        ON trg.EVETP_SOURCE_ID = src.EVETP_SOURCE_ID
            AND trg.EVETP_SOURCE_SYSTEM_ID = src.EVETP_SOURCE_SYSTEM_ID
            AND trg.EVETP_SOURCE_SYS_ORIGIN = src.EVETP_SOURCE_SYS_ORIGIN
            AND trg.EVETP_VALID_TO = src.EVETP_VALID_TO 
        WHERE src.EVETP_SOURCE_ID IS NULL;

        -- Step 6: Close Old Records in Target
        UPDATE gap_catalog.ads_owner.EVENT_TYPES 
        SET EVETP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
            EVETP_UPDATE_PROCESS_KEY = CAST(p_process_key AS BIGINT),
            EVETP_CURRENT_FLAG = 'N', 
            EVETP_VALID_TO = DATEADD(DAY, -1, CAST(p_load_date AS DATE))
        WHERE EVETP_CURRENT_FLAG = 'Y'
            AND EVETP_VALID_TO = DATE('3000-01-01')
            AND CONCAT(evetp_key, '.', evetp_valid_from) IN (
                SELECT tech_rid 
                FROM gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MEP_ADS_MAP_SCD_DIFF 
                WHERE tech_rid IS NOT NULL
            );

        -- Step 7: Insert Changed Records
        INSERT INTO gap_catalog.ads_owner.EVENT_TYPES (
            EVETP_KEY, 
            EVETP_SOURCE_ID, 
            EVETP_SOURCE_SYSTEM_ID, 
            EVETP_SOURCE_SYS_ORIGIN, 
            EVETP_DESC, 
            EVETP_EVENT_TABLE_NAME, 
            EVETP_VALID_FROM, 
            EVETP_VALID_TO, 
            EVETP_CURRENT_FLAG, 
            EVETP_DELETED_FLAG, 
            EVETP_INSERTED_DATETIME, 
            EVETP_INSERT_PROCESS_KEY, 
            EVETP_UPDATED_DATETIME, 
            EVETP_UPDATE_PROCESS_KEY, 
            EVETP_TARGET, 
            EVETP_DATA_PATH
        )
        SELECT 
            EVETP_KEY, 
            EVETP_SOURCE_ID, 
            EVETP_SOURCE_SYSTEM_ID, 
            EVETP_SOURCE_SYS_ORIGIN, 
            EVETP_DESC, 
            EVETP_EVENT_TABLE_NAME, 
            CAST(p_load_date AS DATE) as EVETP_VALID_FROM, 
            DATE('3000-01-01') as EVETP_VALID_TO, 
            'Y' as EVETP_CURRENT_FLAG, 
            tech_del_flg as EVETP_DELETED_FLAG, 
            CURRENT_TIMESTAMP() as EVETP_INSERTED_DATETIME, 
            CAST(p_process_key AS BIGINT) as EVETP_INSERT_PROCESS_KEY, 
            CURRENT_TIMESTAMP() as EVETP_UPDATED_DATETIME, 
            CAST(p_process_key AS BIGINT) as EVETP_UPDATE_PROCESS_KEY, 
            EVETP_TARGET, 
            EVETP_DATA_PATH
        FROM gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MEP_ADS_MAP_SCD_DIFF
        WHERE tech_new_rec = 'N';

        -- Step 8: Insert New Records
        INSERT INTO gap_catalog.ads_owner.EVENT_TYPES (
            EVETP_KEY, 
            EVETP_SOURCE_ID, 
            EVETP_SOURCE_SYSTEM_ID, 
            EVETP_SOURCE_SYS_ORIGIN, 
            EVETP_DESC, 
            EVETP_EVENT_TABLE_NAME, 
            EVETP_VALID_FROM, 
            EVETP_VALID_TO, 
            EVETP_CURRENT_FLAG, 
            EVETP_DELETED_FLAG, 
            EVETP_INSERTED_DATETIME, 
            EVETP_INSERT_PROCESS_KEY, 
            EVETP_UPDATED_DATETIME, 
            EVETP_UPDATE_PROCESS_KEY, 
            EVETP_TARGET, 
            EVETP_DATA_PATH
        )
        SELECT 
            xxhash64(uuid()) AS EVETP_KEY,
            EVETP_SOURCE_ID, 
            EVETP_SOURCE_SYSTEM_ID, 
            EVETP_SOURCE_SYS_ORIGIN, 
            EVETP_DESC, 
            EVETP_EVENT_TABLE_NAME, 
            CAST(p_load_date AS DATE) as EVETP_VALID_FROM, 
            DATE('3000-01-01') as EVETP_VALID_TO, 
            'Y' as EVETP_CURRENT_FLAG, 
            tech_del_flg as EVETP_DELETED_FLAG, 
            CURRENT_TIMESTAMP() as EVETP_INSERTED_DATETIME, 
            CAST(p_process_key AS BIGINT) as EVETP_INSERT_PROCESS_KEY, 
            CURRENT_TIMESTAMP() as EVETP_UPDATED_DATETIME, 
            CAST(p_process_key AS BIGINT) as EVETP_UPDATE_PROCESS_KEY, 
            EVETP_TARGET, 
            EVETP_DATA_PATH
        FROM gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MEP_ADS_MAP_SCD_DIFF
        WHERE tech_new_rec = 'Y';
        CALL gap_catalog.log.complete_performance_test(
            process_key => p_process_key,
            pstatus => 'COMPLETED'
        );
    END;
    
END;